<a href="https://colab.research.google.com/github/josooik/DeepLearning/blob/main/19_Kaggle_dataset_Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Training Set: Self Driving Cars
https://www.kaggle.com/roydatascience/training-car

### 데이터 읽어오기

In [ ]:
! gdown https://drive.google.com/u/0/uc?id=1X8OFchk1JsJS2j66nos-E77MG_KoHXuv&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=1X8OFchk1JsJS2j66nos-E77MG_KoHXuv
To: /content/archive.zip
100% 650M/650M [00:16<00:00, 38.3MB/s]


KeyboardInterrupt: ignored

In [ ]:
!unzip -q archive.zip

### 데이터 경로 설정

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

path = './self driving car training data/data/'

PATH_IMG = sorted(glob(os.path.join(path, 'IMG/*.jpg')))
PATH_CSV = os.path.join(path, 'driving_log.csv')

print("* PATH_IMG :", PATH_IMG)
print("* PATH_IMG 갯수 :", len(PATH_IMG))

print("\n")

print("* PATH_CSV :", PATH_CSV)
print("* PATH_CSV 갯수 :", len(PATH_CSV))

### 로그파일 경로

In [ ]:
data_frame = pd.read_csv(PATH_CSV)
data_frame

### 라벨값 히스토그램
* steering
* throttle
* break
* speed

In [ ]:
streering = data_frame[data_frame.columns[3]].values
throttle = data_frame[data_frame.columns[4]].values
brak = data_frame[data_frame.columns[5]].values
speed = data_frame[data_frame.columns[6]].values

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(12, 4))

plot_list = [steering, throttle, brak, speed]
plot_name = ['steering', 'throttle', 'brak', 'speed' ]

for i in range(len(plot_list)):
  plt.subplot(1, len(plot_list), i+1)
  plt.title(plot_name[i])
  plt.hist(plot_list[i])

plt.show()

### Data Generator with augmentation

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

class DataGenerator():
  def __init__(self, img_size, batch_size):
    self.img_size = img_size
    self.batch_size = batch_size

  def path2arr(self, path, mask=False):
    # img 일때
    if not mask:
      # path -> img
      img = load_img(path, color_mode='rgb', target_size=(self.img_size, self.img_size))
      # img -> arr
      arr = img_to_array(img, dtype='uint8')

    return arr
  
  def load_datasets(self, img_paths, mask_paths):
    img_arrs = []

    img_arr = self.path2arr(img_path)
    img_arrs.append(img_arr)

    return np.array(img_arrs)

  def preprocess(self, img_arr, mask_arr):
    img_arr = (img_arr / 127.5) -1

    return img_arr

  def tf_preprocess(self, img_arr, mask_arr):
    img_arr = (tf.cast(img_arr, tf.float32) / 127.5)

    return img_arrs

  def augment(slef, img, mask):
    # 밝기랜덤조절
    img = tf.image.random_brightness(img, max_delta=0.2)

    img = tf.image.random_contrast(img, 0.5, 2.0)
    img = tf.image.random_saturation(img, 0.75, 1.25)

    img = tf.clip_by_value(img, -1, 1)

    # 좌우랜덤반전 / 상하랜덤반전
    ds = tf.image.random_flip_left_right(ds)
    #ds = tf.image.random_flip_up_down(ds)
    
    # 이미지 사이즈 증가(padding)
    #pd_size = 50
    #img = tf.image.resize_with_crop_or_pad(img, self.img_size + pad_size, self.img_size + pad_size)

    # 랜덤크롭
    #ds = tf.cncat((img, mask), axis=2)
    #ds = tf.image.random_crop(ds, size=[self.img_size, self.img_size, 4])
    #img = (256, 256, 3)

    return img

  def generate(self, img_paths, mask_paths, aug=False):
    # 기존 Numpy 이용한 데이터셋 처리
    img_arrs, mask_arrs = self.load_datasets(img_paths, mask_paths)

    # tensorflow dataset 라이브러리 사용 여기서부터는 Numpy가 아니라 Tensor 이용
    dataset = tf.data.Dataset.from_tensor_slices((img_arrs, mask_arrs))
    # shuffle
    dataset = dataset.shuffle(1)
    dataset = dataset.map(self.tf_preprocess, num_parallel_calls=tf.data.AUTOTUNE) # 전처리, num_parallel_calls=tf.data.AUTOTUNE -> 병렬처리
    
    if aug:
      dataset.map(self.augment, num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(self.batch_size, drop_remainder=True) # batch 사이즈 지정

    print(dataset)

    return dataset

In [ ]:
train_x = []
train_y = []

path = './self driving car training data/data/IMG'
stear_adjust_factor = 0.2
IMAGE_HEIGHT = 100 
IMAGE_WIDTH = 100

for i in range(no_of_examples):
    
    for choice in range(3):
        
        if choice == 0: #Center
            img = cv2.imread(os.path.join(path, center[i].split('IMG/')[1]))
            steering_angle = steering[i]

        elif choice == 1: #Left
            img = cv2.imread(os.path.join(path, left[i].split('IMG/')[1]))
            steering_angle = steering[i] + stear_adjust_factor

        elif choice == 2: #Right
            img = cv2.imread(os.path.join(ipath, right[i].split('IMG/')[1]))
            steering_angle = steering[i] - stear_adjust_factor
        
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:,:,1]
        img = cv2.resize(img,(IMAGE_HEIGHT,IMAGE_WIDTH))
        
        train_x.append(img)
        train_y.append(steering_angle)
        
        flip_img,steering_angle = random_flip(img,steering_angle)
                
        train_x.append(flip_img)
        train_y.append(steering_angle)
        

train_x = np.array(train_x)
train_x = np.reshape(train_x,[train_x.shape[0],train_x.shape[1],train_x.shape[2],1])

train_y = np.array(train_y)
train_y = np.reshape(train_y,[train_y.shape[0],1])

print(train_x.shape)
print(train_y.shape)

# 1. 데이터셋 확보 후 알려주시고

# 2. 모델 만들고나서 알려주시고

### 모델

In [ ]:
# 전이학습
# 모델 가져온후
# 끝부분에 Flatten + Dense + 마지막에 값하나 예측